# Installion of Libraries

In [ ]:
#Install necessary libraries
# Run the following commands in your terminal or Jupyter Notebook if not already installed
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install pillow

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from tkinter import Tk, Label, Button, filedialog, Canvas
from PIL import ImageTk, Image
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Dataset import

In [ ]:
# Paths to dataset
train_dir = 'Dataset/train'  
test_dir = 'Dataset/test'   

In [ ]:
# Parameters
img_size = (224, 224)
batch_size = 32
epochs = 20



In [ ]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)




In [ ]:
# Loading data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)



# Model architecture building code

In [ ]:
# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])



In [ ]:
# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train Model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)



In [ ]:
# Save Model
model.save('breast_cancer_detection_model.h5')
print("Model saved as 'breast_cancer_detection_model.h5'")



In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from tkinter import Tk, Label, Button, filedialog, Canvas
from PIL import ImageTk, Image




In [ ]:
# Load the trained model
model = tf.keras.models.load_model('breast_cancer_detection_model.h5')


In [ ]:
# Define class labels
class_labels = {0: "Benign", 1: "Malignant"}


In [ ]:
# Function to preprocess and predict
def predict_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions for batch
    prediction = model.predict(img_array)[0][0]
    class_label = class_labels[int(prediction > 0.5)]
    confidence = prediction if prediction > 0.5 else 1 - prediction
    return class_label, confidence
    
    
    

In [ ]:
# Function to plot relevant graphs
def plot_graphs(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot accuracy
    ax1.plot(history['accuracy'], label='Train Accuracy')
    ax1.plot(history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    # Plot loss
    ax2.plot(history['loss'], label='Train Loss')
    ax2.plot(history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()

    plt.tight_layout()
    plt.show()
    

In [ ]:
def upload_and_predict():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if not file_path:
        return

    # Display image
    img = Image.open(file_path)
    img = img.resize((300, 300), Image.Resampling.LANCZOS)  # Updated line
    img = ImageTk.PhotoImage(img)
    canvas.create_image(150, 150, image=img)
    canvas.image = img

    # Predict and display results
    class_label, confidence = predict_image(file_path)
    result_label.config(text=f"Prediction: {class_label}\nConfidence: {confidence * 100:.2f}%")

In [ ]:
# Initialize Tkinter GUI
root = Tk()
root.title("Breast Cancer Detection")
root.geometry("500x600")


In [ ]:
# Add widgets
Label(root, text="Breast Cancer Detection System", font=("Arial", 18)).pack(pady=10)

canvas = Canvas(root, width=300, height=300, bg="white")
canvas.pack(pady=20)

upload_button = Button(root, text="Upload Image", command=upload_and_predict, font=("Arial", 14))
upload_button.pack(pady=10)

result_label = Label(root, text="", font=("Arial", 14))
result_label.pack(pady=20)

In [ ]:
# Optionally, plot graphs if training history is available
try:
    # Load training history (if saved as a dictionary)
    history = {
        "accuracy": [0.8, 0.85, 0.9, 0.92, 0.95],
        "val_accuracy": [0.78, 0.83, 0.88, 0.9, 0.93],
        "loss": [0.5, 0.4, 0.3, 0.2, 0.1],
        "val_loss": [0.55, 0.45, 0.35, 0.25, 0.2]
    }
    plot_button = Button(root, text="Show Training Graphs", command=lambda: plot_graphs(history), font=("Arial", 14))
    plot_button.pack(pady=10)
except Exception as e:
    print("No training history available to plot graphs.")
    

In [ ]:
# Run the GUI
root.mainloop()


